In [1]:
#Import module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
#read from web (prioritise on June)
june = [180602, 180609, 180616, 180623, 180630]
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
df_list = []
for week in june:
    df_list.append(pd.read_csv(url.format(week)))
df_mta = pd.concat(df_list)

In [5]:
print (df_mta.head(5))
print (df_mta.tail(5))
print (df_mta.shape)

    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  05/26/2018  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  05/26/2018  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  05/26/2018  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  05/26/2018  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  05/26/2018  16:00:00   

      DESC  ENTRIES  \
0  REGULAR  6633898   
1  REGULAR  6633916   
2  REGULAR  6633928   
3  REGULAR  6634015   
4  REGULAR  6634173   

   EXITS                                                                 
0                                            2248498                     
1                                            2248500                     
2                                            2248517                     
3                                            2248577                     
4                                       

In [6]:
df_mta.DATE.unique()

array(['05/26/2018', '05/27/2018', '05/28/2018', '05/29/2018',
       '05/30/2018', '05/31/2018', '06/01/2018', '06/02/2018',
       '06/03/2018', '06/04/2018', '06/05/2018', '06/06/2018',
       '06/07/2018', '06/08/2018', '06/09/2018', '06/10/2018',
       '06/11/2018', '06/12/2018', '06/13/2018', '06/14/2018',
       '06/15/2018', '06/16/2018', '06/17/2018', '06/18/2018',
       '06/19/2018', '06/20/2018', '06/21/2018', '06/22/2018',
       '06/23/2018', '06/24/2018', '06/25/2018', '06/26/2018',
       '06/27/2018', '06/28/2018', '06/29/2018'], dtype=object)